In [1]:
import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS")
elif torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Using GPU")
else:
    device = torch.device("cpu")
    print("No GPU/MPS available, falling back to CPU.")

from diffusion import init_pipe
from utils import cosine_similarity, double_plot, get_colors
from dataloaders.mesh_container import MeshContainer
from dino import init_dino
from sam2_setup import init_sam2
from pytorch3d.io import load_objs_as_meshes
from utils import compute_features
import numpy as np

Using GPU


/workspace/miniconda3/envs/diff3f/lib/python3.10/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [6]:
num_views = 25
H = 512
W = 512
tolerance = 0.004
use_normal_map = True
num_images_per_prompt = 1
bq = True
use_sam = False
use_only_diffusion = False
use_diffusion = True
is_tosca = False

In [5]:
# sam_model = init_sam2(device)
pipe = init_pipe(device)
dino_model = init_dino(device)

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'pipeline_controlnet_img2img.StableDiffusionControlNetImg2ImgPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .
Downloading: "https://github.com/facebookresearch/dinov2/zipball/main" to /root/.cache/torch/hub/main.zip
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attentio

In [7]:
# please download the data from here into the data folder: https://drive.google.com/drive/folders/1C6lFfCbwQqxlvUE8niVfbeIzeblCnXcx?usp=share_link
# pip install gdown && gdown --folder https://drive.google.com/drive/folders/1C6lFfCbwQqxlvUE8niVfbeIzeblCnXcx?usp=share_link
source_file_path = "data/SHREC20b_lores/models/cow.obj"
source_prompt = "cow"

target_file_path = "data/SHREC20b_lores/models/camel_a.obj"
target_prompt = "camel"


# Check if MeshContainer is available
try:
    source_mesh = MeshContainer().load_from_file(source_file_path)
    target_mesh = MeshContainer().load_from_file(target_file_path)
except NameError:
    # Fallback to load_objs_as_meshes if MeshContainer is not available
    source_mesh = load_objs_as_meshes([source_file_path], device=device)
    target_mesh = load_objs_as_meshes([target_file_path], device=device)

In [8]:
sam_model = None
f_source = compute_features(device, sam_model, dino_model, pipe, source_mesh, source_prompt, num_views, H, W, tolerance)
f_target = compute_features(device, sam_model, dino_model, pipe, target_mesh, target_prompt, num_views, H, W, tolerance)

Starting batch_render with num_views=25, H=512, W=512
Rendering completed successfully
Starting batch_render with num_views=25, H=512, W=512
Rendering completed successfully


/workspace/miniconda3/envs/diff3f/lib/python3.10/site-packages/torch/functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /home/conda/feedstock_root/build_artifacts/libtorch_1729805341246/work/aten/src/ATen/native/TensorShape.cpp:3609.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████| 50/50 [04:00<00:00,  4.81s/it]


Number of missing features:  0
Copied features from nearest vertices
Time taken in mins:  4.261463729540507
Starting batch_render with num_views=25, H=512, W=512
Rendering completed successfully
Starting batch_render with num_views=25, H=512, W=512
Rendering completed successfully


100%|██████████| 50/50 [04:15<00:00,  5.10s/it]


Number of missing features:  31
Copied features from nearest vertices
Time taken in mins:  4.553315603733063


In [18]:
s = cosine_similarity(f_source.to(device),f_target.to(device))
s = torch.argmax(s, dim=0).cpu().numpy()
np.save('predicted_mapping.npy', s)

cmap_source = get_colors(source_mesh.vert); cmap_target = cmap_source[s]
double_plot(source_mesh,target_mesh,cmap_source,cmap_target)

In [20]:
from eval import evaluate_meshes

# Call the evaluation function
avg_error, accuracy, distances = evaluate_meshes(
    source_file_path = "data/SHREC20b_lores/models/cow.obj",
    target_file_path = "data/SHREC20b_lores/models/camel_a.obj",
    source_gt_path = 'data/SHREC20b_lores_gts/cow.mat',
    target_gt_path = 'data/SHREC20b_lores_gts/camel_a.mat',
    mapping_path = 'predicted_mapping.npy', 
    debug=False
)

print(f"Average correspondence error (err): {avg_error:.6f}")
print(f"Correspondence accuracy (acc, γ=1%): {accuracy:.6f}")

Average correspondence error (err): 0.138106
Correspondence accuracy (acc, γ=1%): 0.729167
